In [ ]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net
from bni_netica.bni_utils import findAllDConnectedNodes

CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")
CauseEffectNet = Net("./nets/outputs/common_cause_effect.neta")

listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

def printNet(net):
    for node in net.nodes():
        print(f"{node.name()} -> {[child.name() for child in node.children()]}")

def is_connected(net, node1, node2):
  relatedNodes = net.node(node1).getRelated("d_connected")
  return any(n.name() == node2 for n in relatedNodes)

def stateIdx(node, spec):
    if isinstance(spec, int):
        return spec
    st = node.state(spec)  # by state NAME
    if st is None:
        st = node.stateByTitle(spec)  # fallback by title
    if st is None:
        raise ValueError(f"Unknown state '{spec}' for node '{node.name()}'")
    return st.stateNum

# Common Cause & Effects

In [ ]:
def _names(nodes):
    return {n.name() for n in nodes}

def ancestors(net, node):
    return _names(net.node(node).getRelated("ancestors,exclude_self"))

def descendants(net, node):
    return _names(net.node(node).getRelated("descendents,exclude_self"))

def get_common_cause(net, node1, node2):
    return ancestors(net, node1) & ancestors(net, node2)

def get_common_effect(net, node1, node2):
    return descendants(net, node1) & descendants(net, node2)


## Test

In [17]:
# net
# A → B → C ← D ← E
# A ← X ← Z → Y → E

print('Common Causes of X and E:', get_common_cause(CauseEffectNet, "X", "E"))
print('Common Effects of B and E:', get_common_effect(CauseEffectNet, "B", "E"))

print('Common Causes of A and E:', get_common_cause(CauseEffectNet, "A", "E"))
print('Common Effects of A and E:', get_common_effect(CauseEffectNet, "A", "E"))

Common Causes of X and E: {'Z'}
Common Effects of B and E: {'C'}
Common Causes of A and E: {'Z'}
Common Effects of A and E: {'C'}


# Check for change of dependancy of X, Y after observing Z

In [ ]:
from contextlib import contextmanager

@contextmanager # release resource automatically after using it
def _temporarily_set_findings(net, findings_dict):
    """findings_dict: {node_name: state_name_or_index}"""
    saved = net.findings()  # current evidence (indices)
    try:
        # Apply new evidence
        for k, v in findings_dict.items():
            node = net.node(k)
            if isinstance(v, int):
                node.finding(v)
            else:
                node.finding(v)  # state(name)
        net.update()
        yield
    finally:
        # Restore previous evidence
        net.retractFindings()
        for k, v in (saved or {}).items():
            net.node(k).finding(v)
        net.update()

def does_Z_change_dependency(net, X, Y, Z):
    """
    Returns (changed: bool, details: {'before': bool, 'after': bool})
    where True means 'd-connected' (dependent), False means 'd-separated' (independent).
    """
    zname = net.node(Z).name()
    saved = net.findings()  # keep current evidence E to restore later
    
    try:
        # ---- BEFORE: dependency without Z ----
        if saved and zname in saved:
            # ensure Z is not observed for the 'before' check
            net.node(zname).retractFindings()
            net.update()
        dep_before = is_connected(net, X, Y)

        # ---- AFTER: dependency under Z observed ----
        with _temporarily_set_findings(net, {zname: 0}): # using state index 0 for Z
            dep_after = is_connected(net, X, Y)

        return (dep_before != dep_after), {"before": dep_before, "after": dep_after}
    finally:
        # Restore original evidence exactly as it was
        net.retractFindings()
        if saved:
            net.findings(saved)
        net.update()

def evidences_block_XY(net, X, Y):
    ans = []
    evidences = findAllDConnectedNodes(net, X, Y)
    # print('evidences:', [e.name() for e in evidences])
    for e in evidences:
        e_name = e.name()
        if e_name != X and e_name != Y and does_Z_change_dependency(net, X, Y, e_name)[0]:
            ans.append(e_name)
    return ans

## Test

In [5]:
netDir = "./nets/"
myNet = Net(netDir+"NF_V1.dne")

print('Net: ', myNet.name())
printNet(myNet)
print()

Net:  NativeFishV1
Rainfall -> ['TreeCond', 'PesticideInRiver', 'RiverFlow']
Drought -> ['TreeCond', 'RiverFlow']
TreeCond -> []
PesticideUse -> ['PesticideInRiver']
PesticideInRiver -> ['FishAbundance']
RiverFlow -> ['FishAbundance']
FishAbundance -> []



In [18]:
print('Net: ', myNet.name())
printNet(myNet)
print()

findingsPest = myNet.node("PesticideInRiver").stateNames()
print('Findings on PesticideInRiver:', findingsPest)

print()

print(evidences_block_XY(myNet, "RiverFlow", "PesticideInRiver"))
# print(does_Z_affect_XY(myNet, X="Rainfall", Y="RiverFlow", Z="FishAbundance"))

# print('does_Z_change_dependency:', end=' ')
print(does_Z_change_dependency(myNet, X="Rainfall", Y="Drought", Z="TreeCond"))


Net:  NativeFishV1
Rainfall -> ['TreeCond', 'PesticideInRiver', 'RiverFlow']
Drought -> ['TreeCond', 'RiverFlow']
TreeCond -> []
PesticideUse -> ['PesticideInRiver']
PesticideInRiver -> ['FishAbundance']
RiverFlow -> ['FishAbundance']
FishAbundance -> []

Findings on PesticideInRiver: ['High', 'Low']

evidences: ['Rainfall', 'RiverFlow', 'PesticideInRiver']
['Rainfall']
(True, {'before': False, 'after': True})


# Prob of X when observe Y

In [ ]:
def _output_distribution(beliefs, node):
    """Pretty print a distribution (list of probabilities) with state names."""
    output = ""
    for i, p in enumerate(beliefs):
        state = node.state(i)
        output += f"  P({node.name()}={state.name()}) = {p:.4f}\n"

    return output

def prob_X_given_Y(net, X=None, Y=None, y_state="Yes"):
    """
    Returns P(X | Y = y_state), net_after_observation
    y_state can be state names (str) or indices (int).
    """
    
    with _temporarily_set_findings(net, {Y: y_state}):
        node_X = net.node(X)
        # beliefs() is P(X | current findings)
        return node_X.beliefs(), net
    
def prob_X_given_YZ(net, X=None, Y=None, y_state="Yes", Z=None, z_state="Yes"):
    """
    Returns P(X = x_state | Y = y_state, Z = z_state), net_after_observation
    x_state, y_state and z_state can be state names (str) or indices (int).
    """
    
    with _temporarily_set_findings(net, {Y: y_state, Z: z_state}):
        node_X = net.node(X)
        # beliefs() is P(X | current findings)
        return node_X.beliefs(), net
    
def get_prob_X_given_Y(net, X=None, Y=None, y_state="Yes"):
    """
    Returns string output of prob_X_given_Y
    """
    beliefs, net_after = prob_X_given_Y(net, X, Y, y_state)
    output = f"P({X} | {Y}={y_state}):\n"
    output += _output_distribution(beliefs, net_after.node(X))
    return output, net_after

def get_prob_X_given_YZ(net, X=None, Y=None, y_state="Yes", Z=None, z_state="Yes"):
    """
    Returns string output of prob_X_given_YZ
    """
    beliefs, net_after = prob_X_given_YZ(net, X, Y, y_state, Z, z_state)
    output = f"P({X} | {Y}={y_state}, {Z}={z_state}):\n"
    output += _output_distribution(beliefs, net_after.node(X))
    return output, net_after

## Test

In [ ]:
out, _ = get_prob_X_given_Y(myNet, X="Rainfall", Y="TreeCond", y_state="Good")
print(out)

out2, _ = get_prob_X_given_YZ(myNet, X="Rainfall", Y="Drought", y_state="Yes", Z="TreeCond", z_state="Good")
print(out2)

P(Rainfall | TreeCond=Good):
  P(Rainfall=Below_average) = 0.0857
  P(Rainfall=Average) = 0.6909
  P(Rainfall=Above_average) = 0.2235

P(Rainfall | Drought=Yes, TreeCond=Good):
  P(Rainfall=Below_average) = 0.0784
  P(Rainfall=Average) = 0.6863
  P(Rainfall=Above_average) = 0.2353



#### Archive codes

In [ ]:
# def joint_prob(net, assignments):
#     """
#     assignments: dict { node_name: state_index or state_name }
#                  (state_name must exist on that node)
#     Returns P(assignments | current findings)
#     """
#     # Build a stable order
#     items = list(assignments.items())

#     # Make node list
#     node_list = g.NewNodeList2_bn(len(items), net.eNet)

#     # States array (Netica 'state_bn' = c_int)
#     states = (state_bn * len(items))()

#     for i, (name, val) in enumerate(items):
#         node = net.node(name)
#         if node is None:
#             raise ValueError(f"Unknown node '{name}'")

#         # Put node pointer into nodelist
#         g.SetNthNode_bn(node_list, i, node.eId)

#         # Resolve state index
#         if isinstance(val, int):
#             s_idx = val
#         else:
#             st = node.state(val)
#             if st is None:
#                 raise ValueError(f"Unknown state '{val}' on node '{name}'")
#             s_idx = st.stateNum

#         states[i] = s_idx

#     # Make sure beliefs are up to date
#     net.update()

#     # Call Netica
#     prob = g.JointProbability_bn(node_list, states)
#     return float(prob)


# def xy_joint_table_by_Z(net, X, Y, Z):
#     nodeZ = net.node(Z)
#     table = {}
#     for zi in range(nodeZ.numberStates()):
#         zname = nodeZ.state(zi).name()
#         with temporarily_set_findings(net, {Z: zi}):
#             # Example: return the joint table for X×Y under Z=zi
#             nodeX, nodeY = net.node(X), net.node(Y)
#             j = {}
#             for xi in range(nodeX.numberStates()):
#                 for yi in range(nodeY.numberStates()):
#                     p = joint_prob(net, {X: xi, Y: yi})
#                     j[(nodeX.state(xi).name(), nodeY.state(yi).name())] = p
#             table[zname] = j
#     return table

# def does_Z_affect_XY(net, X, Y, Z, tol=1e-12):
#     # the result table (a Python dict) that stores all the joint probabilities you computed for each value of Z.
#     # dict { z_state_name: { (x_state_name, y_state_name): prob, ... }, ... }
#     tbl = xy_joint_table_by_Z(net, X, Y, Z) 
    
#     # Compare the first table to the others
#     it = iter(tbl.values())
#     first = next(it)
#     for other in it:
#         for k in first:
#             if abs(first[k] - other[k]) > tol:
#                 return True, tbl  # differs for at least one state of Z
#     return False, tbl

# relatedToPesticideUse = myNet.node("PesticideInRiver").parents()
# print('parents of PesticideInRiver:', [n.name() for n in relatedToPesticideUse])
# set1 = set(n.name() for n in relatedToPesticideUse)

# relatedToPesticideUse = myNet.node("RiverFlow").parents()
# print('parents of RiverFlow:', [n.name() for n in relatedToPesticideUse])
# set2 = set(n.name() for n in relatedToPesticideUse)

# print('Intersection:', set1.intersection(set2))

# relatedToPesticideUse = myNet.node("PesticideInRiver").getRelated("d_connected")
# print('relatedTo PesticideInRiver:', [n.name() for n in relatedToPesticideUse])
# set1 = set(n.name() for n in relatedToPesticideUse)

# relatedToPesticideUse = myNet.node("RiverFlow").getRelated("d_connected")
# print('relatedTo RiverFlow:', [n.name() for n in relatedToPesticideUse])
# set2 = set(n.name() for n in relatedToPesticideUse)

# print('Intersection:', set1.intersection(set2))

# print()

# print('[d-connected nodes between PesticideInRiver and RiverFlow]')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideInRiver", "RiverFlow")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and Rainfall]')

# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and FishAbundance]')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "FishAbundance")
# print([n.name() for n in nodes])

# print('[d-connected nodes between PesticideUse and Rainfall | PesticideInRiver]')
# myNet.node('PesticideInRiver').finding('Low')
# nodes = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall")
# print([n.name() for n in nodes])
# myNet.retractFindings()

# print('[d-connected nodes between PesticideUse and Rainfall | FishAbundance]')
# myNet.node('FishAbundance').finding('Low')
# arcs = bni_utils.findAllDConnectedNodes(myNet, "PesticideUse", "Rainfall", {"arcs": True})
# print([arc for arc in arcs])

Common Causes of A and E: {'Z'}
Common Effects of A and E: {'C'}
